## Imports

In [58]:
#DO NOT CHANGE THESE LINES
using Suppressor
@suppress begin
    using DataFrames
    using LazyJSON
    using GLM 
    using MLJ 
    using MLJBase
    using CSV
    using Serialization
    using MLJScientificTypes
    using CategoricalArrays
    using JLD2
    using StatsBase
    using StatsModels
end

## Paths

In [59]:
# DO NOT CHANGE THESE LINES
ROOT_DIR = dirname(pwd())
MODEL_INPUTS_OUTPUTS = joinpath(ROOT_DIR, "model_inputs_outputs")
INPUT_DIR = joinpath(MODEL_INPUTS_OUTPUTS, "inputs")
INPUT_SCHEMA_DIR = joinpath(INPUT_DIR, "schema")
DATA_DIR = joinpath(INPUT_DIR, "data")
TRAIN_DIR = joinpath(DATA_DIR, "training")
TEST_DIR = joinpath(DATA_DIR, "testing")
MODEL_PATH = joinpath(MODEL_INPUTS_OUTPUTS, "model")
MODEL_ARTIFACTS_PATH = joinpath(MODEL_PATH, "artifacts")
OHE_ENCODER_FILE = joinpath(MODEL_ARTIFACTS_PATH, "ohe.jld2")
PREDICTOR_DIR_PATH = joinpath(MODEL_ARTIFACTS_PATH, "predictor")
PREDICTOR_FILE_PATH = joinpath(PREDICTOR_DIR_PATH, "predictor.jld2")
IMPUTATION_FILE = joinpath(MODEL_ARTIFACTS_PATH, "imputation.ser")
TOP_CATEGORIES = joinpath(MODEL_ARTIFACTS_PATH, "top_categories.ser")
TARGET_LEVELS = joinpath(MODEL_ARTIFACTS_PATH, "target_levels.ser")
TARGET_MAPPING = joinpath(MODEL_ARTIFACTS_PATH, "target_mapping.ser")

if !isdir(MODEL_ARTIFACTS_PATH)
    mkdir(MODEL_ARTIFACTS_PATH)
end
if !isdir(PREDICTOR_DIR_PATH)
    mkdir(PREDICTOR_DIR_PATH)
end

### Reading the schema
The schema contains metadata about the datasets. We will use the scehma to get information about the type of each feature (NUMERIC or CATEGORICAL) and the id and target features, this will be helpful in preprocessing stage.

In [60]:
# Reading a schema from a JSON file and extracting features
file_name = first(filter(x -> endswith(x, "json"), readdir(INPUT_SCHEMA_DIR)))
schema_path = joinpath(INPUT_SCHEMA_DIR, file_name)
schema_string = read(schema_path, String)  # Read file content as a string
schema = LazyJSON.parse(schema_string)  # Parse using LazyJSON

features = schema["features"]

# Identifying numeric, categorical, and nullable features
numeric_features = String[]
categorical_features = String[]
nullable_features = String[]

for f in features
    if f["dataType"] == "CATEGORICAL"
        push!(categorical_features, f["name"])
    else
        push!(numeric_features, f["name"])
    end
    if f["nullable"]
        push!(nullable_features, f["name"])
    end
end

# Extracting ID and target features
id_feature = schema["id"]["name"]
target_feature = schema["target"]["name"]
target_classes = schema["target"]["classes"]

if length(target_classes) == 2
    negative_class = target_classes[1]
    positive_class = target_classes[2]
end


"1"

### Reading training data

In [61]:
# Find the first CSV file in the TRAIN_DIR
file_name = first(filter(x -> endswith(x, ".csv"), readdir(TRAIN_DIR)))
file_path = joinpath(TRAIN_DIR, file_name)

# Read the CSV file into a DataFrame
df = CSV.File(file_path) |> DataFrame

# Display the first few rows of the DataFrame
first(df, 5)


,id,destination,passenger,weather,temperature,time,coupon
,Int64,String15,String15,String7,Int64,String7,String31
1,10760,No Urgent Place,Partner,Sunny,55,2PM,Bar
2,12114,No Urgent Place,Alone,Sunny,80,10AM,Restaurant(<20)
3,4412,No Urgent Place,Alone,Sunny,80,10AM,Bar
4,10869,Home,Alone,Snowy,30,10PM,Coffee House
5,4808,No Urgent Place,Alone,Sunny,80,10AM,Bar


## Data Preprocessing
Data preprocessing is very important before training the model, as the data may contain missing values in some cells. Moreover, most of the learning algorithms cannot work with categorical data, thus the data has to be encoded.

In this section we will impute the missing values and encode the categorical features. Afterwards the data will be ready to train the model.

##### Imputing missing data
> The median value will be used to impute missing values of the numeric features and the mode will be used to impute categorical features.

##### You can add your own preprocessing steps such as:
<ul>
<li>Normalization</li> <br>
<li>Outlier removal</li><br>
<li>Dropping or adding features</li><br>
</ul>

### Important note:
<p> 
Saving the values used for imputation during training step is crucial. These values will be used to impute missing data in the testing set. This is very important to avoid the well known problem of data leakage. During testing, you should not make any assumptions about the data in hand, alternatively anything needed during the testing phase should be learned from the training phase. This is why we are creating a dictionary of values used during training to reuse these values during testing.
</p>


In [62]:
# Imputing missing data
imputation_values = Dict{String, Any}()

for column in nullable_features
    if column in numeric_features
        value = median(skipmissing(df[!, column]))
    else
        value = mode(skipmissing(df[!, column]))
    end
    df[!, column] = coalesce.(df[!, column], value)
    imputation_values[column] = value
end

# Serialize the imputation_values dictionary to a binary file
open(IMPUTATION_FILE, "w") do io
    serialize(io, imputation_values)
end


In [63]:
# Saving the id and target columns in a different variable
ids = df[!, Symbol(id_feature)]
target = df[!, Symbol(target_feature)]

# Dropping the id and target from the DataFrame
select!(df, Not([Symbol(id_feature), Symbol(target_feature)]))

,destination,passenger,weather,temperature,time,coupon
,String15,String15,String7,Int64,String7,String31
1,No Urgent Place,Partner,Sunny,55,2PM,Bar
2,No Urgent Place,Alone,Sunny,80,10AM,Restaurant(<20)
3,No Urgent Place,Alone,Sunny,80,10AM,Bar
4,Home,Alone,Snowy,30,10PM,Coffee House
5,No Urgent Place,Alone,Sunny,80,10AM,Bar
6,Work,Alone,Rainy,55,7AM,Carry out & Take away
7,Work,Alone,Sunny,55,7AM,Restaurant(<20)
8,No Urgent Place,Friend(s),Sunny,80,6PM,Restaurant(<20)
9,No Urgent Place,Friend(s),Snowy,30,10AM,Carry out & Take away


##### Encoding Categorical features
<p>
The id column is just an identifier for the training example, so we will exclude it during the encoding phase.<br>
Target feature will be label encoded in the next step.
</p>


In [64]:
# Function to get top 10 categories
function get_top_categories(df, features, n=10)
    top_cats = Dict()
    for feature in features
        col_data = df[!, feature]
        category_counts = countmap(col_data)
        sorted_categories = sort(collect(category_counts), by=x->x[2], rev=true)
        
        # Take minimum between n and the number of unique categories
        num_categories = min(n, length(sorted_categories))
        
        top_cats[feature] = [x[1] for x in sorted_categories[1:num_categories]]
    end
    return top_cats
end

# Get top10 categories for specific features
top_categories = get_top_categories(df, categorical_features)

# Function to one-hot encode only the top 10 categories
function one_hot_top_categories!(df, top_categories)
    for (feature, top_cats) in top_categories
        if length(top_cats) == 2  # Handle the binary case
            # Assuming the first category in top_cats is treated as 'true'
            new_col_name = "$(feature)_binary"
            df[!, new_col_name] = df[!, feature] .== top_cats[1]
        else  # Handle the general case
            for cat in top_cats
                new_col_name = "$(feature)_$(cat)"
                df[!, new_col_name] = df[!, feature] .== cat
            end
        end
        select!(df, Not(Symbol(feature)))  # Drop the original feature column
    end
end

# Apply one-hot encoding
one_hot_top_categories!(df, top_categories)

# Serialize the imputation_values dictionary to a binary file
open(TOP_CATEGORIES, "w") do io
    serialize(io, top_categories)
end


### Encoding target feature

In [65]:
encoded_target = ifelse.(target .== positive_class, 1, 0)
mapping = Dict(0 => negative_class, 1 => positive_class)

open(TARGET_MAPPING, "w") do io
    serialize(io, mapping)
end

open(TARGET_LEVELS, "w") do io
    serialize(io, levels(target))
end

16

### Training the Classifier
We choose Logistic Regression model, but feel free to try your own and compare the results.

In [66]:
predictor_columns = names(df)
X = Matrix(df[!, predictor_columns])
model = glm(X, encoded_target, Bernoulli(), LogitLink())

# Save the model
open(PREDICTOR_FILE_PATH, "w") do io
    serialize(io, model)
end
